# Modelizacion con sklearn para detección de fraude

## Explorando los datos

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set() # Sobreescribe los parámetros de matplotlib
plt.rcParams['figure.figsize'] = [10, 5]


In [2]:
## Leemos los datos 
fraude = pd.read_csv('../data/01_datos_4_training_cut.txt', sep='|', nrows =500000)
fraude = fraude.sample(frac=1, random_state = 1) 

### Exploramos
print(fraude.head())
print(fraude.info())

          IDTX     BIN   CIUDAD COD_COMERCIO  COD_MONEDA  COLA  DISPOSITIVO  \
352806  352807  360324  17001MA    012461042         170    -1            2   
417824  417825  455986      INT    013139530         978    -1            2   
469847  469848  589514   5001ME    012800165         170    -1            2   
407746  407747  589514  68307GI    012333910         170    -1            2   
469848  469849  455986  11001BO    013029111         170    -1            2   

        ENTRYMODE ESTADO             FECHATRX  ... MES      NOMBRE_CIO  PAIS  \
352806         51     17  2015-01-04 11:04:40  ...   1    CRA 22 17-11    CO   
417824         51    NaN  2015-01-04 18:14:31  ...   1    HOTEL AGUMAR    ES   
469847         51     05  2015-01-05 10:56:17  ...   1  CRA 66A 34A-25    CO   
407746         51     68  2015-01-04 17:12:21  ...   1             NaN    CO   
469848         12    CUN  2015-01-05 10:56:18  ...   1             NaN    CO   

       PSCONDITION RESPUESTA  TERMINAL  TIPO

## Ejercicio

Calcular:

- La proporción de transacciones fraudulentas en la base de datos y almacenarla en la variable prevalencia
- La proporción del dinero estafado y compararla con los datos del Informe del BCE y almacenarla en la variable proporcion_dinero

> ¿Qué conclusión sacamos?

In [3]:
### BEGIN SOLUTION


### END SOLUTION




In [4]:
print(prevalencia)
print(proporcion_dinero)

NameError: name 'prevalencia' is not defined

In [ ]:
assert prevalencia == sum(fraude.REPORTE_DE_FRAUDE == 'SI')/len(fraude.REPORTE_DE_FRAUDE), 'No has calculado bien la prevalencia'
assert proporcion_dinero == sum(fraude.VALOR_TRX[fraude.REPORTE_DE_FRAUDE == 'SI'])/sum(fraude.VALOR_TRX), 'El dinero defraudado no es el correcto'
print('Bien! Buen trabajo')

# Masajeando los datos

Por nuestro conocimiento experto, sabemos que todas las columnas son __categóricas__ salvo:

- IDTX (es simplemente un índice que podríamos tirar ya que pandas tiene su propio índice)
- FechaTrx (Hora y día de la transacción)
- Valor_TRX (esta sí es numérica)

Sin embargo, muchas de ellas están codificadas con números, creando una falsa apariencia. Hay que convertirlas. 

### Transformando las columnas a sus tipos correctos

Para ello, tendremos que:

1. Eliminar la columna 'IDTX'
2. Convertir la FECHATRX a `datetime` con `pd.to_datetime`
3. Convertir todas las columnas del dataframe a tipo `category` con el método `.astype('category')` salvo la FECHATRX y la VALOR_TRX.


In [ ]:


fraude.drop(['IDTX'], axis = 1, inplace=True)
fraude.FECHATRX = pd.to_datetime(fraude.FECHATRX)

columnas_sin_cambios = ['IDTX', 'FECHATRX','VALOR_TRX']

for columna in fraude.columns:
    if columna not in columnas_sin_cambios:
        fraude[columna] = fraude[columna].astype('category')



In [ ]:
fraude.info()

Veamos ahora cuántos valores diferentes tienen las columnas categóricas. __¿Qué observamos?__

In [ ]:
predictores_cat = list(fraude.select_dtypes(['category']).columns)
for columna in predictores_cat:
    print(columna, len(fraude[columna].cat.categories))

Miremos a ver cómo andamos de Nas

In [ ]:
nulos = fraude.isnull()
suma_nulos = nulos.sum()
ordenados = suma_nulos.sort_values(ascending=False)

print(ordenados)

Salta a la vista que el número de NAs de PSCONDITION y COD_COMERCIO son casi iguales... ¿Qué puede querer decir esto?

In [ ]:
print(len(fraude.COD_COMERCIO[fraude.PSCONDITION.isnull()]))
sum(fraude.COD_COMERCIO[fraude.PSCONDITION.isnull()].isnull())

### Problemas que tenemos con estos datos:

- Las NAs: hay que imputar los valores para poder usar esas observaciones en los algoritmos de aprendizaje. ¿Cómo?
- Las variables categóricas: los algoritmos de ML en general sklearn sólo aceptan variables numéricas. Hay que convertirlas. ¿Cómo?
    - Hay columnas con 300K categorías diferentes. Es inviable usar un one-hot-encoding para ellas
- La única variable numérica tiene una distribución muy sesgada que se adapta mal a nuestros algoritmos
- Y esto como mínimo... Podríamos pensar otras transformaciones de las columnas, generar nuevas columnas derivadas, etc. 



> Como podemos ver, antes de entrenar un algoritmo (¿cuál???), es necesario realizar diversas acciones de preprocesado sobre los datos. Pero para poder realmente estimar con fiabilidad el error del modelo hemos de tener mucho cuidado para que __no  haya filtraciones entre el conjunto de train y el de test__.

> Las operaciones de preprocesado son un terreno abonado para que se produzcan estas filtraciones.

Eso por un lado, pero...

# La gran pregunta

> # ¿Cómo diseño el experimento?

- ¿Qué tengo que buscar?
- ¿Cómo sé si mi clasificador es bueno o no?
- ¿Cómo optimizo el algoritmo?
- ¿Cómo preproceso los datos?
- ¿Cómo sé si va a funcionar cuando le pase otros datos?
- ¿Va a poder ser implementado en el mundo real? 
- ...

# Un poco de calentamiento

Recordemos que `sklearn` __solo puede operar con arrays de numpy__. También puede operar con dataframes siempre y cuando __sean numéricos__. Cualquier operación de sklearn __devuelve siempre arrays de numpy__.

sklearn además __no sabe cómo manejar los NAs ni los infs__. Hay que tener cuidado para que en nuestras columnas no aparezcan este tipo de valores

Por lo tanto, si queremos entrenar un modelo sencillo con los datos de fraude, sólo podemos usar de primeras la columna VALOR_TRX. Vamos a ello.

In [ ]:
# Dividamos primero en train y test para ver los ejemplos. 
# Esto no se hará así cuando estemos con un modelo real, 
# pero viene bien para ver qué está ocurriendo
# Vamos a entrenar una regresión logística con la única variable numérica que tenemos

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import linear_model

X = fraude[['VALOR_TRX']]
y = fraude['REPORTE_DE_FRAUDE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)


mod_log_1 = linear_model.LogisticRegression()
mod_log_1.fit(X_train, y_train)

## `estimators`

> Acabamos de entrenar un __estimador de sklearn__ con el método ```.fit(X, y)```. Este es uno de los dos objetos fundamentales de su API (el otro son los `transformers`)

> El otro método que podemos aplicarle es el ```.predict(X)``` para que nos de los valores estimados de la ```y```



In [ ]:
y_predict = mod_log_1.predict(X_test)
print(y_predict)

Veamos qué tal funciona mi modelo:

In [ ]:
print(mod_log_1.score(X_test, y_test))

Qué buen resultado!!! ... ¿O no?? Veamos si en realidad esto está haciendo algo:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(pd.crosstab(y_test, y_predict, 
                  rownames=['Reales'], colnames=['Predicciones'], margins=True))
print(classification_report(y_test, y_predict))

Uno de los métodos posibles para tener en cuenta el sesgo de las variables es usar algoritmos que puedan pesar las clases. Afortunadamente, la simple regresión logística puede hacerlo pasandole el argumento `class_weight = 'balanced'`.
### Ejercicio

Entrenar un modelo de regresión logística que equilibre las clases. Mostrar los resultados del modelo sobre el conjunto de test.

In [ ]:

### BEGIN SOLUTION

## Entrenar un modelo de regresión logística que equilibre las clases


### END SOLUTION



In [ ]:

## Predecir sobre el conjunto de test
y_predict = mod_log_2.predict(X_test)

## Mostrar los restultados e INTERPRETARLOS
print(mod_log_2.score(X_test, y_test))

print(pd.crosstab(y_test, y_predict, 
                  rownames=['Reales'], colnames=['Predicciones'], margins=True))
print(classification_report(y_test, y_predict))

In [ ]:
from sklearn.utils.validation import check_is_fitted
check_is_fitted(mod_log_2)
assert mod_log_2.get_params()['class_weight'] == 'balanced', 'No es el modelo pedido'
print('Bien! Buen trabajo')

Como puede verse, ahora el algoritmo ha pasado a clasificar muchísimas transacciones como fraude!

> __¿Por qué? ¿Cómo decide si una transacción es fraude o no?__ 

> __¿Tiene sentido usar accuracy como la métrica a optimizar?__

Es por la asignación de probabilidad. Ahora, ¿nos interesa ese umbral..?
Cambiémoslo!

# Adaptando la función de coste a nuestro problema

`sklearn` no solo permite predecir la CLASE sino también la PROBABILIDAD.

Usaremos esta funcionalidad para crear una función de coste _que sea adecuada a nuestro problema (y a su futura implementación)_

> No olvidemos que para _validar, seleccionar el algoritmo y estimar su error_ __tenemos que hacerlo usando una función de coste específica__

In [ ]:
y_pred_prob = mod_log_1.predict_proba(X_test)[:,1]
y_pred_prob

Vamos a definir nuestro scoring rule. Tiene una interpretación bastante intuitiva: es el número de alarmas que el operador puede levantar considerando los medios que tiene disponibles para investigarlas.
Obviamente alarmaremos las transacciones que mi algoritmo crea que son las más sospechosas (las de probabilidad más alta). Construyamos un dataframe que contenga estas probabilidades y la etiqueta de fraude de la transacción

In [ ]:
resultados = pd.DataFrame({'Prob':y_pred_prob, 'Label':y_test.values})
print(resultados.head())
print(resultados.info())

Ordenémoslo descendentemente

In [ ]:
resultados.sort_values('Prob', axis=0, ascending=False, inplace=True)
resultados.reset_index(inplace=True)
print(resultados.head())

Y fijemos un número de alarmas (por ejemplo, el 1% del total de transacciones)

In [ ]:
alarmas = int(0.01*len(y_test))
print('Casos Analizados:{}'.format(len(y_test)))
print('Alarmas:{}'.format(alarmas))
print('Cazados:{}'.format(sum(resultados[0:alarmas].Label=='SI')))
print('Fraude total en el conjunto:{}'.format(sum(resultados.Label == 'SI')))

Este scoring lo vamos a usar repetidas veces, así que metámoslo en una función. 
Para usar un número con una referencia directa en el problema, como número de alarmas voy a poner el número de transacciones fraudulentas en el test. Para fijar otro valor ya dependería de las necesidades y capacidad operativa del cliente.

In [ ]:
def score(mod, X_test, y_test, alarm = None):
    if not alarm:
        alarm = sum(y_test == 'SI')/len(y_test)
    y_pred_prob = mod.predict_proba(X_test)[:,1]
    resultados = pd.DataFrame({'Prob':y_pred_prob, 'Label':y_test.values})
    resultados.sort_values('Prob', axis=0, ascending=False, inplace=True)
    resultados.reset_index(inplace=True)
    alarmas = int(alarm*len(y_test))
    print('Casos Analizados:{}'.format(len(y_test)))
    print('Alarmas:{}'.format(alarmas))
    print('Cazados:{}'.format(sum(resultados[0:alarmas].Label=='SI')))
    print('Fraude total en el conjunto:{}'.format(sum(resultados.Label == 'SI')))
    return



score(mod_log_1, X_test, y_test, alarm=0.01)


Para tratar de mejorar esto tendremos que introducir más variable e implementar de un modo riguroso __las operaciones de preprocesado__

# Preprocesando los datos

La distribución de los valores de las variables predictoras puede tener una importancia crucial en el resultado de nuestro algoritmo de aprendizaje. Investiguemos un poco
¿Cómo es la distribución de nuestra variable X?

In [ ]:
print(X_train.mean())
print(X_train.std())

`sklearn` ofrece diversas operaciones de preprocesado que nos ayudarán a mejorar el rendimiento de los clasificadores. En la API de `sklearn` se realizan mediante los `tranformers`

## `transformers`

- Son objetos de `sklearn` que tienen dos métodos asociados: `fit` y `transform`
- `fit` ajusta los parámetros con los datos que le pasemos
- `transform` transforma los datos que le pasemos usando los parámetros obtenidos con el método `fit`

In [ ]:
from sklearn.preprocessing import StandardScaler

escalado = StandardScaler()
escalado.fit(X_train)
transformado = escalado.transform(X_train)

print(transformado.mean())
print(transformado.std())

In [ ]:
print(escalado.mean_)
print(escalado.scale_)

El problema principal con la variable VALOR_TRX no era su escala sino el _sesgo_ que tiene.


In [ ]:
sns.distplot(X_train)
plt.show()

Se puede arreglar con la transformación de BoxCox. 

In [ ]:
import scipy as sc
from scipy.stats import boxcox

X_bc, param = boxcox((np.array(X)+1).flatten())

sns.distplot(X_bc)
plt.show()
print(param)

- El objeto boxcox devuelve dos elementos: los datos transformados y el parámetro de la transformación utilizado
- Añadimos el +1 para evitar el cero, que no puede ser transformado

Las últimas versiones de `sklearn` ya llevan implementada esta transformación en el transformer `PowerTransformer`

In [ ]:
from sklearn.preprocessing import PowerTransformer

PT = PowerTransformer(method='yeo-johnson')
PT.fit(X_train)
transformado = PT.transform(X_train)

sns.distplot(transformado)
plt.show()


# La API de sklearn

Podemos interpretar un modelo de sklearn como compuesto por _transformaciones de los datos_ en serie o en paralelo que desembocan en un _algoritmo de aprendizaje automático_. 

- Los _transformers_ se entrenan con el método `.fit` y transforman los datos con el método `.transform`
- Los _estimators_ se entrenan con el método `.fit` y predicen con el método `.predict`


El problema que nos vamos a encontrar ahora es que las transformaciones que apliquemos en el conjunto de train tienen que ser realizadas separadamente de las del conjunto de test, y en este último, deben ser replicadas __con los mismos parámetros__ que los obtenidos en el train. 

> ¿Por qué? 



Así mismo, cualquier operación de preprocesado que apliquemos tiene que ser validada para ver si hace mejorar el algoritmo. Esto supondría realizar una validación cruzada con las operaciones/sin ellas

> ¿Cómo hacemos esto?



Y hemos visto que hay que hacer, sólo para una columna, ya dos operaciones de preprocesado (BoxCox y Scaler)... ¡Y aún nos quedan un montón de columnas!

> ¿Cómo se puede hacer esto ordenadamente, controlando los errores y evitando que se filtre información entre el conjunto de train y el de test?




# __USANDO TUBERÍAS (PIPES)__
 

Es una manera de encadenar una serie de transformers con un algoritmo de modo que actúe como "uno solo"

<img src="../images/mario_bros.jpg" width="900px;" align="center"/>

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


# Definimos por dónde van a pasar los datos

steps = [('Imputador', SimpleImputer(strategy='median')),
         ('BoxCox',  PowerTransformer(method='yeo-johnson')),
         ('Escalador', StandardScaler()),
         ('predictor', linear_model.LogisticRegression(class_weight = 'balanced'))]

# Montamos la cañería
pipe = Pipeline(steps)

# Y la entrenamos!!!
pipe.fit(X_train,y_train)


Lo maravilloso de la idea es que los transformers guardan _los parámetros aprendidos con los datos de train y los aplican tal cual a los de test_ , evitando así posibles filtraciones que nos llevarían al overfitting

In [ ]:
y_predict = pipe.predict(X_test)

print(pipe.score(X_test, y_test))

print(pd.crosstab(y_test, y_predict, 
                  rownames=['Reales'], colnames=['Predicciones'], margins=True))
print(classification_report(y_test, y_predict))

score(pipe, X_test, y_test)

In [ ]:
score(pipe, X_test, y_test, alarm=0.01)

__PREGUNTA: ¿Por qué obtenemos lo mismo que antes de aplicar el BoxCox con nuestra métrica pero sin embargo la matriz de confusión o el accuracy ha cambiado considerablemente?__



# Probemos con otro algoritmo

Veamos ahora por qué la API de sklearn es una auténtica maravilla.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Definimos por dónde van a pasar los datos

steps = [('Imputador', SimpleImputer(strategy='median')),
         ('BoxCox',  PowerTransformer(method='yeo-johnson')),
         ('Escalador', StandardScaler()),
         ('predictor', RandomForestClassifier(n_jobs=-1))]

# Montamos la cañería
pipe = Pipeline(steps)

# Y la entrenamos!!!
pipe.fit(X_train,y_train)

score(pipe, X_test, y_test)


In [ ]:
score(pipe, X_test, y_test, alarm=0.01)

## Ejercicio:
Probar con vuestro algoritmo de clasificación favorito.

In [ ]:
    ### BEGIN SOLUTION
   
    ### END SOLUTION

In [ ]:
score(pipe, X_test, y_test, alarm=0.1)

# Mirando dentro de la tubería
Aunque se comporte como un bloque, podemos ejecutar la tubería paso por paso si eso fuera necesario 

In [ ]:
steps

In [ ]:
tmp = steps[0][1].fit_transform(X_train)
tmp

In [ ]:
tmp = steps[1][1].fit_transform(tmp)
tmp

### Pregunta: ¿Qué hará la siguiente instrucción? ¿Hemos usado la tubería al completo?

In [ ]:
 steps[3][1].fit(tmp, y_train)

# Metamos las variables categóricas

Para simplificar un poco, vamos por el momento a olvidarnos de las columnas más problemáticas (las categóricas con muchos valores) y vamos a centrarnos en las más "amables" para ilustrar los conceptos.

In [ ]:

# Definimos los predictores
predictores_num = [fraude.columns[-2]]
predictores_cat = list(fraude.columns[i] for i in [3,4,5,6,10,13,14,15,17])

# Definimos los vectores de predictores y la respuesta
y = fraude['REPORTE_DE_FRAUDE']
X = fraude[predictores_num + predictores_cat]

X.info()

# Dividimos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)


# ¿Cómo codificamos las variables categóricas?

> Respuesta: no existe un método _neutro_


Los más habituales son:

- OneHotEncoder - para los predictores: crea una columna por categoría y ala codifica con 0 y 1
- Label Binarizer – para la respuesta: crea una columna por categoría y la codifica con 0 y 1
- OrdinalEncoder – para predictores: sustituye categorías por enteros 1,2,3,.. Impone un orden implícito
- LabelEncoder – para la respuesta: sustituye categorías por enteros 1,2,3,,.. Impone un orden implícito

> La diferencia entre _predictor_ y _respuesta_ es la _dimensión_ de la array

In [ ]:

data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = np.array(data)
print(values)


## One-hot encoder

La representación estándar para variables categóricas es la de one-hot-encoding, fácil de ver con un ejemplo. 

Supongamos que tenemos una variable que toma tres valores `Rojo`, `Azul`  y `Verde`. Basta pues con crear tres nuevas variables numéricas (las llamadas dummy variabels) y hacer la siguiente transformación (recordad cuando teníamos variables categóricas en la clase de regresión lineal):

* `Rojo` -> (1, 0, 0).
* `Azul` -> (0, 1, 0).
* `Verde` -> (0, 0, 1).

Mediante pandas, basta usar `get_dummies`:

In [ ]:
pd.get_dummies(values)

En `sklearn` (que es lo que nos ocupa), se hace con `OneHotEncoder`

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
oh_encoded = one_hot_encoder.fit_transform(values.reshape(-1,1))
print(oh_encoded)

Como podemos ver, __la salida del transformer on hot encoder es una matriz dispersa__

In [ ]:
type(oh_encoded)

In [ ]:
oh_encoded.toarray()

## Ventajas del OH-encoding

- __Ventajas__: No introduce un orden en la variable; las mantiene en un espacio ortogonal
- __Inconvenientes__: La dimensionalidad puede dispararse puesto que añadimos un predictor por cada categoría

# ¿Cómo evitamos que la dimensión de los datos se dispare?

- Realizar un PCA de las columnas one-hot
- Usar una codificación con un orden implícito (`OrdinalEncoder`)
- Reducir el número de categorías a las más $n$ más populares y aglomerar el resto en una única categoría
- Usar una codificación por frecuencia
- Usar una codificación por variable objetivo (`Target Encoding`)

# OrdinalEncoder 

Es posible representar variables categóricas asignándo un número natural a cada valor diferente mediante `OrdinalEncoder`. Sin embargo, esto puede no ser lo ideal puesto que estamos introduciendo una estructura en los datos que antes no tenían.
Eso sí: puede ser muy adecuado para variables categóricas que tengan un orden intrínseco

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

print(values)

ordinal_encoder = OrdinalEncoder(categories=[['cold', 'warm', 'hot']])
values_encoded = ordinal_encoder.fit_transform(values.reshape(-1,1))
print(values_encoded)


- __Ventajas__: Ocupa poca memoria
- __Inconvenientes__ (o ventajas!!): Ordena los valores de la variable, creando una estructura que antes no había

# ¿Y cómo preprocesamos las columnas de manera específica?

> Está claro que vamos a tener que aplicar diferentes operaciones de preprocesado dependiendo del tipo de columna

En nuestro caso:

- Imputar NAs (la media a la variable numérica y la moda a las categóricas)
- Transformar y estandarizar la columna numérica
- Codificar algunas de las columnas categóricas a dummies (con OneHotEncoder)
- Codificar otras columnas categóricas de otra manera


![Alt Text](https://i2.wp.com/adhikary.net/wp-content/uploads/2019/03/Screenshot-from-2019-03-23-09-22-51.png?resize=768%2C432&ssl=1)

## `ColumnTransformer`

Para poder tratar de diferente manera en la cañería las columnas categóricas de las numéricas, usaremos la función de sklearn `ColumnTransformer`.

In [ ]:
# Definimos los predictores
# predictores_num = [fraude.columns[-2]]
# predictores_cat = list(fraude.columns[i] for i in [4,5,6,7,11,14,15,16,18])

from sklearn.compose import ColumnTransformer

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('BoxCox',  PowerTransformer(method='yeo-johnson')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocesado = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, predictores_num),
        ('cat', categorical_transformer, predictores_cat)])

Ensamblemos este preprocesado al algoritmo

In [ ]:
# Definimos la tubería

steps = [('feat_prepro', preprocesado), 
         ('predictor', RandomForestClassifier(n_jobs=-1))]

pipe = Pipeline(steps)

pipe.fit(X_train, y_train)

# Estimamos el error del modelo

In [ ]:
y_predict = pipe.predict(X_test)

print(pipe.score(X_test, y_test))

print(pd.crosstab(y_test, y_predict, 
                  rownames=['Reales'], colnames=['Predicciones'], margins=True))
print(classification_report(y_test, y_predict))

score(pipe, X_test, y_test)


In [ ]:
score(pipe, X_test, y_test, alarm=0.01)



# Ejercicio 

Ampliar la tubería anterior para que:
- Primero extraiga las 20 primeras componentes de una descomposición en valores principales truncado ([+INFO](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html))
- Y luego amplíe las features añadiendo términos de interacción hasta segundo
orden ([+INFO](https://scikit-learn.org/stable/modules/preprocessing.html#generating-polynomial-features)
)  
- Usar un `linear_model.LogisticRegression` como clasificador


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier

### BEGIN SOLUTION



### END SOLUTION



In [ ]:
pipe_2 = Pipeline(steps)

pipe_2.fit(X_train, y_train)

y_predict = pipe_2.predict(X_test)

print(pipe_2.score(X_test, y_test))

print(pd.crosstab(y_test, y_predict, 
                  rownames=['Reales'], colnames=['Predicciones'], margins=True))
print(classification_report(y_test, y_predict))

score(pipe_2, X_test, y_test)

In [ ]:
score(pipe_2, X_test, y_test, alarm = 0.01)

> Usando las operaciones de preprocesado adecuadas, podemos hacer que un algoritmo sencillo mejore claramente

# ¿Cómo seguir mejorando a partir de aquí?

- Utilizar las features que aún no hemos usado. Para ellos, habrá que buscar algún método para codificarlas.
- Ingeniería de Features: que tendremos que integrar también en la pipeline y en el proceso de validación. 
- Ajustar los hiperparáetros del algoritmo


# Necesitamos aprender a ensamblar elementos personalizados en la tubería